In [31]:
import sys
from tensorflow import keras
import sklearn
import tensorflow as tf
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import time

In [ ]:
def load_data():
    # 28*28 - 0~255로 진행 - keras 데이터로 (sklearn 데이터 x)
    (X_train_full, y_train_full), (
    X_test, y_test) = tf.keras.datasets.mnist.load_data()  # 훈련 세트는 60,000개의 흑백 이미지입니다. 각 이미지의 크기는 28x28 픽셀입니다:
    
    # 기존 데이터는 60000:10000 -> 80:20 비율 위해 40000:10000으로 설정
    X_train_full=X_train_full[:40000]
    y_train_full=y_train_full[:40000]
    
    return X_train_full, y_train_full, X_test, y_test


def data_normalization(X_train_full, y_train_full, X_test, divide):

    # 정규화 여부 divide로 결정
    X_valid, X_train = X_train_full[:5000] / divide, X_train_full[5000:] / divide
    y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
    X_test = X_test / divide

    return X_valid, X_train, y_valid, y_train, X_test

In [40]:
def makemodel(X_train, y_train, X_valid, y_valid, weight_init, Dropout=False, epoch=10):
    model = keras.models.Sequential()

    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    model.add(keras.layers.Dense(300, kernel_initializer=weight_init, activation="relu"))
    if Dropout:
        model.add(tf.keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(100, kernel_initializer=weight_init, activation="relu"))
    if Dropout:
        model.add(tf.keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(10, kernel_initializer=weight_init, activation="softmax"))

    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="sgd",
                  metrics=["accuracy"])

    # 시간 측정
    tb_hist = keras.callbacks.TensorBoard(log_dir='./graph', histogram_freq=0, write_graph=True, write_images=True)
    start = time.time()
    history = model.fit(X_train, y_train, epochs=epoch,
                        validation_data=(X_valid, y_valid), callbacks=[tb_hist])
    print("time :", time.time() - start)
    return model, history


def evalmodel(model, history, X_test, y_test):
    model.evaluate(X_test, y_test)

    X_new = X_test[:3]
    y_proba = model.predict(X_new)
    y_proba.round(2)



# 1. 가중치 초기화 

In [ ]:
def main():
    # train:test = 80:20
    X_train_full, y_train_full, X_test, y_test = load_data()
    X_valid, X_train, y_valid, y_train, x_test = data_normalization(X_train_full, y_train_full, X_test, 255.)
    
    print('Random 가중치 초기화\n')
    model_Random, history_Random = makemodel(X_train, y_train, X_valid, y_valid, 'RandomNormal')
    evalmodel(model_Random, history_Random, x_test, y_test)
    
    print('\n Xavier 가중치 초기화\n')
    model_Xavier, history_Xavier = makemodel(X_train, y_train, X_valid, y_valid, 'glorot_uniform')
    evalmodel(model_Xavier, history_Xavier, x_test, y_test)

main()

Random 가중치 초기화

Epoch 1/10
1094/1094 [==============================] - 4s 4ms/step - loss: 1.2923 - accuracy: 0.6854 - val_loss: 0.5139 - val_accuracy: 0.8718
Epoch 2/10
1094/1094 [==============================] - 4s 3ms/step - loss: 0.4319 - accuracy: 0.8814 - val_loss: 0.3453 - val_accuracy: 0.9050
Epoch 3/10
1094/1094 [==============================] - 4s 3ms/step - loss: 0.3399 - accuracy: 0.9031 - val_loss: 0.2923 - val_accuracy: 0.9176
Epoch 4/10
1094/1094 [==============================] - 4s 3ms/step - loss: 0.2961 - accuracy: 0.9144 - val_loss: 0.2641 - val_accuracy: 0.9270
Epoch 5/10
1094/1094 [==============================] - 4s 4ms/step - loss: 0.2659 - accuracy: 0.9236 - val_loss: 0.2376 - val_accuracy: 0.9330
Epoch 6/10
1094/1094 [==============================] - 4s 3ms/step - loss: 0.2408 - accuracy: 0.9309 - val_loss: 0.2188 - val_accuracy: 0.9414
Epoch 7/10
1094/1094 [==============================] - 4s 3ms/step - loss: 0.2194 - accuracy: 0.9377 - val_loss: 0.2020

### 1. 결과 비교  
 Random과 Xavier 가중치 초기화 방식을 모두 진행했을 때 Xavier를 사용한 경우가  
 train, val, test 모든 데이터와 전체 학습 기간에 대해 더 낮은 loss와 높은 accuracy를 가진다.  
 시간의 경우는 Random보다는 Xavier 방식이 조금 더 걸리는 것을 알 수 있다.  
 정리하면, MNIST 데이터셋에 대해 Xavier 가중치 초기화 방식이 학습에 더 효과적이다.

# 2. 정규화 여부

In [34]:
def main():
    # train:test = 80:20
    X_train_full, y_train_full, X_test, y_test = load_data()
    
    print('0~255 Ver\n') # divide를 1로 설정하여 0~255 그대로 유지
    X_valid, X_train, y_valid, y_train, x_test = data_normalization(X_train_full, y_train_full, X_test, 1.)
    model_255, history_255 = makemodel(X_train, y_train, X_valid, y_valid, 'glorot_uniform')
    evalmodel(model_255, history_255, x_test, y_test)
    
    print('\n 0~1 정규화 Ver\n') # divide를 255로 설정하여 0~1로 정규화
    X_valid, X_train, y_valid, y_train, x_test = data_normalization(X_train_full, y_train_full, X_test, 255.)
    model_normalization, history_normalization = makemodel(X_train, y_train, X_valid, y_valid, 'glorot_uniform')
    evalmodel(model_normalization, history_normalization, x_test, y_test)

main()

0~255 Ver

Epoch 1/10
1094/1094 [==============================] - 4s 4ms/step - loss: 33310.0664 - accuracy: 0.1129 - val_loss: 2.6032 - val_accuracy: 0.1126
Epoch 2/10
1094/1094 [==============================] - 4s 3ms/step - loss: 2.3146 - accuracy: 0.1143 - val_loss: 2.3712 - val_accuracy: 0.1126
Epoch 3/10
1094/1094 [==============================] - 3s 3ms/step - loss: 2.3011 - accuracy: 0.1143 - val_loss: 2.3712 - val_accuracy: 0.1126
Epoch 4/10
1094/1094 [==============================] - 4s 3ms/step - loss: 2.3011 - accuracy: 0.1143 - val_loss: 2.3711 - val_accuracy: 0.1126
Epoch 5/10
1094/1094 [==============================] - 3s 3ms/step - loss: 2.3011 - accuracy: 0.1143 - val_loss: 2.3710 - val_accuracy: 0.1126
Epoch 6/10
1094/1094 [==============================] - 4s 3ms/step - loss: 2.3011 - accuracy: 0.1143 - val_loss: 2.3711 - val_accuracy: 0.1126
Epoch 7/10
1094/1094 [==============================] - 4s 4ms/step - loss: 2.3011 - accuracy: 0.1143 - val_loss: 2.3712 

### 2. 정규화 여부 결과
 0~255로 정규화하지 않은 경우와 0~1로 정규화한 경우를 비교해보면  
 극심할 정도로 정규화를 진행한 쪽이 모든 데이터의 전체 학습 기간에 대해 loss가 낮고, accuracy는 높다.  
 에포크가 10으로 작지만, time도 정규화를 진행한 쪽이 살짝 덜 걸렸다.  
 정리하면, 전체적으로 0~1로 정규화를 진행한 쪽이 학습에 매우 효과적이었다.

# 3. dropout 여부

In [ ]:
def main():
    # train:test = 80:20
    X_train_full, y_train_full, X_test, y_test = load_data()
    
    print('Dropout 미적용 Ver\n') # 에포크 50
    X_valid, X_train, y_valid, y_train, x_test = data_normalization(X_train_full, y_train_full, X_test, 255.)
    model_normal, history_normal = makemodel(X_train, y_train, X_valid, y_valid, 'glorot_uniform', False, 50)
    evalmodel(model_normal, history_normal, x_test, y_test)
    
    print('\n Dropout 적용 Ver\n') # 에포크 50
    X_valid, X_train, y_valid, y_train, x_test = data_normalization(X_train_full, y_train_full, X_test, 255.)
    model_dropout, history_dropout = makemodel(X_train, y_train, X_valid, y_valid, 'glorot_uniform', True, 50)
    evalmodel(model_dropout, history_dropout, x_test, y_test)

main()

Dropout 미적용 Ver

Epoch 1/50
1094/1094 [==============================] - 5s 5ms/step - loss: 0.7709 - accuracy: 0.8084 - val_loss: 0.3569 - val_accuracy: 0.9050
Epoch 2/50
1094/1094 [==============================] - 4s 4ms/step - loss: 0.3362 - accuracy: 0.9046 - val_loss: 0.2847 - val_accuracy: 0.9206
Epoch 3/50
1094/1094 [==============================] - 5s 4ms/step - loss: 0.2768 - accuracy: 0.9204 - val_loss: 0.2380 - val_accuracy: 0.9352
Epoch 4/50
1094/1094 [==============================] - 5s 4ms/step - loss: 0.2417 - accuracy: 0.9312 - val_loss: 0.2128 - val_accuracy: 0.9438
Epoch 5/50
1094/1094 [==============================] - 4s 4ms/step - loss: 0.2151 - accuracy: 0.9388 - val_loss: 0.1935 - val_accuracy: 0.9486
Epoch 6/50
1094/1094 [==============================] - 4s 4ms/step - loss: 0.1939 - accuracy: 0.9441 - val_loss: 0.1787 - val_accuracy: 0.9522
Epoch 7/50
1094/1094 [==============================] - 5s 4ms/step - loss: 0.1762 - accuracy: 0.9493 - val_loss: 0.163

### 3. dropout 여부 결과
dropout을 진행한 경우 에포크가 낮은 초기 단계에서  
학습하는 뉴런 수가 줄기에 학습 속도와 효율이 떨어진다.  
즉 초기에서는 dropout을 적용하지 않은 경우에 비해  
loss가 높고 정확도가 낮아지는 결과를 보인다.  
다시 정리하면, 과잉적합 되지 않는 에포크 수가 낮은 경우에는  
오히려 드롭아웃을 사용하지 않거나 비율을 낮게 하는 것이 결과가 더 좋게 나온다.  
dropout은 모델이 과잉적합되어 일반화 성능이 떨어질 때 효과적이다.  
즉 모델이 어느정도 epoch 정도부터 과잉적합이 되는지 정확히 파악하지  
못하는 학습 진행 단계에서 epoch를 크게 돌려놓고, dropout을 적용해  
학습을 진행하며 과잉적합되는 시기를 늦추며 더 좋은 성능의  
모델을 결과를 얻을 수 있다.  

아래 결과는 50에포크를 수행한 결과이다.  
dropout을 적용한 경우에 val, test 데이터에 대해 적용하지 않은 경우에 비해  
loss가 낮고, acc가 소폭 높은 것을 확인할 수 있다. 
즉 일반화 성능이 dropout을 적용했을 때 더 좋다.  
추가로 train의 loss와 acc를 살펴봤을 때 dropout을 적용하지 않은 경우  
train 데이터에 대해 너무 잘 학습되어지는 과잉적합이 의심되는 단계에 있음을  
알 수 있다. dropout을 적용한 경우는 덜 과잉적합 되었음을 알 수 있다.  

결과적으로 위에서 말했던 것처럼 epoch가 커질경우 과잉적합을  
방지하기 위해 dropout을 사용하는 것이 학습성능에 더 효과적이다.  


##### dropout 미적용
1094/1094 [==============================] - 5s 4ms/step - loss: 0.0158 - accuracy: 0.9982 - val_loss: 0.0853 - val_accuracy: 0.9762
time : 227.20604634284973
313/313 [==============================] - 1s 3ms/step - loss: 0.0952 - accuracy: 0.9742
1/1 [==============================] - 0s 55ms/step  



##### dropout 적용
1094/1094 [==============================] - 5s 5ms/step - loss: 0.0454 - accuracy: 0.9869 - val_loss: 0.0706 - val_accuracy: 0.9784
time : 259.8462555408478
313/313 [==============================] - 1s 3ms/step - loss: 0.0731 - accuracy: 0.9782
1/1 [==============================] - 0s 55ms/step